In [ ]:
import pandas as pd
import regex as re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


## BWM Demo for Lecture

Lets read in the two datasets - both are from the BiWeekly Monitoring Study (BWM).  

One has oberservations at the child level, the other at the Household level. 



In [ ]:
child = "https://github.com/UCB-Econ-148/econ148-sp24/raw/main/lab/lab03/BWM_child_EVDvars.dta"
hh = "https://github.com/UCB-Econ-148/econ148-sp24/raw/main/lab/lab03/BWM_HH_EVDvars.dta"

In [ ]:
wg_df = pd.read_stata(child) 
wg_df

In [ ]:
hh_wg = pd.read_stata(hh) 
hh_wg

## Let's look at dates of the surveys

In [ ]:
Dates_df = hh_wg.groupby('bwm_round')['interview_date'].first().reset_index()
Dates_df.head(20)

## Pivot Table  number of households in each round


In [ ]:
hh_wg.pivot_table(index='bwm_round', values='a1_hh_id', aggfunc='count')

## Pivot Table  number of children in each round


In [ ]:
wg_df.pivot_table(index='bwm_round', values='a1_hh_id', aggfunc='count')

##  Lets look at the variables

In [ ]:
hh_wg.columns

In [ ]:
column_names_list = list(hh_wg.columns)
print(column_names_list)


## Lets looks for variables that begin with g
### Using Regex syntax 

In [ ]:
# Pandas String Methods
gvars_ps = hh_wg.loc[:, hh_wg.columns.str.match(r'^[gG]\d')]
gvars_ps


In [ ]:
# Filtering with regex

#gvars = hh_wg.filter(regex='[gG]')
#gvars = hh_wg.filter(regex='^[gG]')
gvars_re = hh_wg.filter(regex='^[gG]\d')
gvars_re

In [ ]:
# Filter for H with regex
hvars_re = hh_wg.filter(regex='^[hH]\d')
hvars_re

In [ ]:
hgvars = hh_wg[['a1_hh_id','bwm_round','g5_current_water_treated', 'h5_chlorine_color', 'h6_chlorine_meter']]
hgvars = hgvars.copy()

### H variables - Chlorine Test 

A reagent tablet is added to a water sample, and the color of the resulting solution is marked on the survey form if visible, and used a device to measure the color as well.  

This is the difference between h5 and h6

In [ ]:
hgvars.loc[:, 'g5xh5']= ((hgvars['g5_current_water_treated']==1) & (hgvars['h5_chlorine_color']>=1)).astype(int)
hgvars.loc[:,'g5xh6']= ((hgvars['g5_current_water_treated']==1) & (hgvars['h6_chlorine_meter']>=0.05)).astype(int)

hgvars

In [ ]:
matches = (hgvars['g5xh5'] == hgvars['g5xh6'])
matches.sum()

In [ ]:
mismatches = (hgvars['g5xh5'] != hgvars['g5xh6'])
mismatches.sum()

In [ ]:
pct = mismatches.sum()/(matches.sum()+mismatches.sum())
pct

### Lets look at the open coding of where water was collected if not collected at the nearest spring

In [ ]:
gvars=gvars_ps.copy()
gvars

In [ ]:
gvars['g4a_collect_other_type'].unique()

In [ ]:
pd.crosstab(gvars['g4a_collect_other_type'], columns='count')

In [ ]:
# Is other still a protected spring?
gvars['protected_count'] = gvars['g4a_collect_other_type'].str.count(r'PROTECTED')
gvars['protected_count'].sum() 

In [ ]:
# Is other an unprotected spring?
gvars['protected_count'] = gvars['g4a_collect_other_type'].str.count(r'UNPROTECTED')
gvars['protected_count'].sum() 

# Let's  make a new variable for water source types


In [ ]:
gvars['other_source'] = (gvars['g4a_collect_other_type'].notna() & gvars['g4a_collect_other_type'].str.strip().ne('')).astype(int)
gvars['other_source']

In [ ]:
gvars['other_source'].value_counts()

In [ ]:
gvars["other_source"] = gvars["other_source"].astype("string")


In [ ]:
gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\brain\s*water\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Rainwater'

How This Works:

-	\brain\s*water\b
-	rain → Matches “rain”
-	\s* → Matches zero or more spaces (RAIN WATER or RAINWATER)
-	water → Matches “water”
-	\b → Ensures it doesn’t mistakenly match something like "brainwater"

In [ ]:

gvars['other_source'].value_counts()

In [ ]:
gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\bprotected\s*spring\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Protected Spring'

gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\bunprotected\s*spring\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Unprotected Spring'

gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\bborehole\b|\bbore\s*hole\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Borehole'

gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\bwell\b|\bshallow\s*well\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Well'


In [ ]:
gvars['other_source'].value_counts()


**Vendor:**
-	Captures "WATER VENDOR", "BOUGHT FROM WATER VENDOR", "FROM WATER VENDORS", etc.
-	Uses \bvendor\b|\bwater\s*vendor\b to account for variations.


**Tap:**
-	Matches "TAP WATER" and "TAP".
-	Uses \btap\s*water\b|\btap\b to allow for "TAP WATER" and "TAP" individually.

In [ ]:
gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\bvendor\b|\bwater\s*vendor\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Vendor'

gvars.loc[gvars['g4a_collect_other_type'].str.contains(r'\btap\s*water\b|\btap\b', flags=re.IGNORECASE, na=False), 'other_source'] = 'Tap'

In [ ]:
gvars['other_source'].value_counts()


In [ ]:
gvars.loc[gvars['other_source']=="1", 'g4a_collect_other_type']

In [ ]:
gvars.loc[gvars['other_source'].ne("0"), 'other_source'].value_counts().plot(kind='bar')